# **Taller Despliegue Flask API**

## PARTE I: Despliegue local

Para esta primera parte vamos a desplegar un modelo de machine learning en una API **para su consumo en local** (como hemos visto en el taller anterior para la base de datos de novelas de ciencia ficción). 

Para ello entrenaremos un modelo, lo guardaremos entrenado, y desarrollaremos una API que permita consumir dicho modelo desde cualquier otra tecnología, pero primero en un servico local.

Para que tengas más contexto, **te presentamos el siguiente caso de uso**.

### Caso de Uso

Una empresa distribuidora de muebles del ámbito nacional pretende utilizar un modelo desarrollado por el departamento de Ciencia de Datos, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos.  

Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc.  

La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas (una API).

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web. El microservicio tiene que cumplir las siguientes características:  

1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad.  

2. Podamos actualizar la base de datos con nuevos registros, una vez conozcamos los valores de venta reales.  

3. Posibilidad de reentrenar el modelo con los nuevos registros.  

### ¿Qué es necesario implementar?  

1. Por simplicidad del ejercicio, la base de datos con la que trabajaremos es un CSV ("data/Advertising.csv")

2. Un modelo sencillo. Nota: Por simplicidad, en el momento de reentrenamiento nos limitaremos a entrenar nuestro regresiór lineal con los últimos datos. No es necesario comprobar si los resultados son mejores o peores, si tenemos datos faltantes, duplicados, outliers, etc.

3. Queremos implementar 3 endpoints:  
    - Endpoint con mensaje local para el acceso al / (home)  

    - Endpoint que dados los valores de inversión en TV, Radio y Periódicos nos diga las ventas esperadas (predicción o inferencia)  
    
    - Endpoint en el que el servidor de la API compruebe si tiene datos nuevos, reentrene el modelo, y vuelva a calcular sus métricas.  
        (Para esto vamos a suponer que la presencia del fichero "data/Advertising_new.csv" equivale a tener datos nuevos)

    **NOTA**: Cuentas con un script de Python (*model.py*) con el código de entrenamiento del modelo ya realizado ya que el objetivo del ejercicio no es desarrollar un modelo de machine learning, sino el diseño de una API con Flask.

### Procedimiento

**#1. Creación del modelo**

Carga el script `model.py`, revisalo y ejecútalo para crear nuestro modelo.

**#2. Creación del script para dar servicio a la API (modo local)**

Completa el script "app_model.py" añadiendo el routing a las siguientes funciones (revísalas antes) (en los comentarios de cada función se describe el endpoint)

```python
def hello(): # Ligado al endopoint "/" o sea el home, con el método GET
    return "Bienvenido a mi API del modelo advertising"
```

```python
def predict(): # Ligado al endpoint '/api/v1/predict', con el método GET
    with open('ad_model.pkl', 'rb') as f:
        model = pickle.load(f)

    tv = request.args.get('tv', None)
    radio = request.args.get('radio', None)
    newspaper = request.args.get('newspaper', None)

    print(tv,radio,newspaper)
    print(type(tv))

    if tv is None or radio is None or newspaper is None:
        return "Args empty, not enough data to predict"
    else:
        prediction = model.predict([[float(tv),float(radio),float(newspaper)]])
    
    return jsonify({'predictions': prediction[0]})
```

Para la función anterior, ¿Cómo crees que espera los argumentos la API? (¿como parámetros en el cuerpo de la petición, como querystring, de otra forma?)

```python
def retrain(): # Ligado al endpoint '/api/v1/retrain/', metodo GET
    if os.path.exists("data/Advertising_new.csv"):
        data = pd.read_csv('data/Advertising_new.csv')

        X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['sales']),
                                                        data['sales'],
                                                        test_size = 0.20,
                                                        random_state=42)

        model = Lasso(alpha=6000)
        model.fit(X_train, y_train)
        rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
        mape = mean_absolute_percentage_error(y_test, model.predict(X_test))
        model.fit(data.drop(columns=['sales']), data['sales'])
        with open('ad_model.pkl', 'wb') as f:
            pickle.dump(model, f)
            
        return f"Model retrained. New evaluation metric RMSE: {str(rmse)}, MAPE: {str(mape)}"
    else:
        return f"<h2>New data for retrain NOT FOUND. Nothing done!</h2>"

```

**#3. Hora de ejecutar, probar varias predicciones, y probar nuestro endpoint de retrain**